## Variable Selection

In [1]:
import pandas as pd
import numpy as np
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import TimeSeriesSplit
from sklearn.metrics import mean_squared_error
from math import sqrt

df = pd.read_csv('C:/Users/dstoc/Documents/Python Scripts/Fixed Income Dashboards/reduced_data_cleaned.csv')
df2 = df.copy()

In [2]:
# Assuming 'Date' is the name of the column containing date information
df2['Date'] = pd.to_datetime(df2['Date'])
df2.set_index('Date', inplace=True)
df2 = df2.replace([np.inf, -np.inf], np.nan).dropna(axis = 1)

# Define the features (independent variables) and target (dependent variable)
features = list(df2)

In [3]:
# Define the lag order (number of lags to create)
lag_order = 52  # Example: Creating lagged values for the past n periods
# Define the initial training size (e.g., 70% of the data)
initial_train_size = int(0.7 * len(df2)) - lag_order
InSamp_length = len(df2) - initial_train_size - lag_order

# Define the number of periods to lead the target variable (1 period in this case)
lead_periods = 1

In [4]:
# redefine the features (independent variables) and target (dependent variable)
features = list(df2)

# Create a new variable representing the 'lead' of the target
df2['YIELD_weekly_percent_change_10 YR_lead'] = df2['YIELD_weekly_percent_change_10 YR'].shift(-lead_periods)

# Drop rows with missing values in the features or target (created by the shift)
df2 = df2.dropna(subset=features + ['YIELD_weekly_percent_change_10 YR_lead'])
df3 = df2.copy()
features = list(df3.columns)[0:(len(df3.columns)-1)]
X = df3[features]
y = df3['YIELD_weekly_percent_change_10 YR_lead']

target_var = 'YIELD_weekly_percent_change_10 YR_lead'

if target_var.lower() in map(str.lower, X.columns):
    print(f"The DataFrame contains the column '{target_var}'.")
else:
    print(f"The DataFrame X does not contain the target variable: '{target_var}'.")


The DataFrame X does not contain the target variable: 'YIELD_weekly_percent_change_10 YR_lead'.


Spearman's rank correlation is often used when the variables being compared may not have a linear relationship or when the assumptions of parametric correlation measures like Pearson's correlation are not met.

In [6]:
df3.iloc[:InSamp_length,].tail()
import pandas as pd

def select_top_variables(df, target_variable, num_variables):
    # Calculate the rank correlation (Spearman's rho) with the target variable
    correlation_matrix = df.corrwith(df[target_variable], method='spearman')
    
    # Sort variables based on absolute correlation values
    sorted_variables = correlation_matrix.abs().sort_values(ascending=False)
    
    # Select the top variables
    selected_variables = sorted_variables.index[:num_variables]
    
    # Extract the correlation values for the selected variables
    selected_correlations = correlation_matrix[selected_variables]
    
    # Create a DataFrame to store the results
    corr_data = pd.DataFrame({
        'Variable': selected_variables,
        'Correlation': selected_correlations
    })
    
    return corr_data

# Assuming df3 is your DataFrame and 'YIELD_weekly_percent_change_10 YR' is the target variable
correlated_variables = select_top_variables(df3.loc[df3.index[:InSamp_length]], 'YIELD_weekly_percent_change_10 YR_lead', 150)

df3.loc[df3.index[:InSamp_length], correlated_variables['Variable']].head()

,YIELD_weekly_percent_change_10 YR_lead,UST_10Y_FO_weekly_percent_change_Non Reportable Shorts,UST_10Y_FU_weekly_percent_change_Non Reportable Longs,UST_10Y_FU_weekly_percent_change_Non Reportable Shorts,UST_10Y_FO_weekly_percent_change_Non Reportable Longs,NATGAS_FO_weekly_percent_change_Noncommercial Long,YEN_FU_weekly_percent_change_Nonreportable Positions Short,UST_10Y_FU_weekly_percent_change_Open Interest,YEN_FO_weekly_percent_change_Nonreportable Positions Short,UST_10Y_FO_weekly_percent_change_Asset Manager Longs,...,UST_BOND_FU_weekly_percent_change_Open Interest,UST_5Y_FU_weekly_percent_change_Asset Manager Shorts,UST_2Y_FO_weekly_percent_change_Total Reportable Longs,NATGAS_FO_weekly_percent_change_Noncommercial Spreads,UST_5Y_FO_weekly_percent_change_Other Reportable Spreads,UST_2Y_FU_weekly_percent_change_Total Reportable Longs,UST_BOND_FU_weekly_percent_change_Leveraged Funds Spreads,CAD_FO_weekly_percent_change_Noncommercial Short,YIELD_weekly_percent_change_10 YR,YEN_FO_weekly_percent_change_Commercial Short
Date,,,,,,,,,,,,,,,,,,,,,
2006-06-25,-0.015296,0.075506,0.039976,0.039968,0.045921,-0.087035,-0.284957,0.096635,-0.277764,0.186905,...,-0.019779,0.007598,0.056834,0.056063,0.189028,0.045936,-0.550656,-0.015993,0.019493,-0.352258
2006-07-02,-0.001942,-0.053162,-0.031766,-0.012708,-0.068202,0.067602,0.098720,0.046394,0.084128,0.061635,...,0.023965,0.013609,-0.004186,-0.022297,-0.128419,0.001315,0.070579,-0.372297,-0.015296,-0.032239
2006-07-09,-0.013619,-0.002422,-0.049494,0.002832,-0.049233,0.014390,-0.108938,-0.026388,-0.095245,-0.015988,...,-0.028735,0.006314,-0.032397,-0.015603,0.095496,-0.031532,-0.091935,-0.122298,-0.001942,0.080360
2006-07-16,-0.003945,-0.022542,0.034387,-0.018527,0.046073,0.034815,-0.140593,-0.011381,-0.142315,0.055753,...,0.012805,0.007803,0.066715,0.009804,0.032502,0.070556,0.307907,1.622942,-0.013619,0.064786
2006-07-23,-0.009901,-0.001156,-0.031302,-0.026084,-0.023064,-0.043627,0.322699,-0.004893,0.299812,0.004527,...,-0.002162,0.012693,0.018339,0.044208,0.311718,0.014198,0.164550,-0.209329,-0.003945,-0.184593


Variance Inflation Factor (VIF) is a tool used in multiple regression analysis to measure the degree of multicollinearity. Multicollinearity occurs when there is a correlation between multiple independent variables in a multiple regression model, which can adversely affect the regression results. The VIF estimates how much the variance of a regression coefficient is inflated due to multicollinearity. The dependent variable is the outcome being acted upon by the independent variables.

In [7]:
from statsmodels.stats.outliers_influence import variance_inflation_factor

def calculate_vif(X):
    vif_data = pd.DataFrame()
    vif_data["Variable"] = X.columns
    vif_data["VIF"] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]
    return vif_data

# Assuming X is your DataFrame containing the selected variables
#vif_results = calculate_vif(df3.iloc[:InSamp_length,[correlated_variables['Variable']]])
vif_results = calculate_vif(df3.loc[df3.index[:InSamp_length], correlated_variables['Variable']])

# Print the variables with high VIF
low_vif_variables = vif_results.sort_values('VIF')[:51]
print("Variables with low VIF:")
print(low_vif_variables)


Variables with low VIF:
                                              Variable        VIF
47      UST_BOND_FU_weekly_percent_change_Dealer Longs   2.105677
139              SOMA_weekly_percent_change_notesbonds   2.368308
5    NATGAS_FO_weekly_percent_change_Noncommercial ...   2.394295
86   UST_10Y_FO_weekly_percent_change_Other Reporta...   2.461150
85     UST_BOND_FO_weekly_percent_change_Dealer Shorts   2.469419
0               YIELD_weekly_percent_change_10 YR_lead   2.515170
21                     SOMA_weekly_percent_change_tips   2.618457
18                    SOMA_weekly_percent_change_total   2.620629
143  NATGAS_FO_weekly_percent_change_Noncommercial ...   2.674793
84   UST_BOND_FU_weekly_percent_change_Other Report...   2.681228
52      UST_10Y_FU_weekly_percent_change_Dealer Shorts   2.883957
102      UST_5Y_FU_weekly_percent_change_Dealer Shorts   3.014391
109  UST_5Y_FU_weekly_percent_change_Leveraged Fund...   3.240387
90   CHF_FU_weekly_percent_change_Nonreportable Pos.

In [33]:
import numpy as np
import pandas as pd
import statsmodels.api as sm
from statsmodels.tsa.stattools import grangercausalitytests
from statsmodels.tsa.stattools import acf, pacf
from statsmodels.tsa.tsatools import lagmat, lagmat2ds

# def generate_fake_data(num_obs=100, num_vars=5, max_lag=3, noise_std=1):
#     np.random.seed(42)
#     data = np.random.randn(num_obs, num_vars + 1)  # +1 for the target variable
#     for i in range(1, num_vars + 1):
#         for lag in range(1, max_lag + 1):
#             data[lag:, i] += 0.5 * data[:-lag, i - 1]
#     data[:, 0] = 0.5 * data[:, 1] + noise_std * np.random.randn(num_obs)  # Target variable
#     return pd.DataFrame(data, columns=['Target'] + [f'Var_{i}' for i in range(1, num_vars + 1)])

def select_variables_granger_causality(data, max_lag_method='aic', num_vars_to_select=2):
    results = []

    for var in data.drop(columns = target_var).columns[0:]:  # Exclude the target variable
        target_series = data[target_var]
        var_series = data[var]

        if max_lag_method == 'acf':
            max_lag = select_lag_acf(var_series)
        elif max_lag_method == 'pacf':
            max_lag = select_lag_pacf(var_series)
        elif max_lag_method == 'aic':
            max_lag = select_lag_aic(target_series, var_series)
        else:
            raise ValueError("Invalid max_lag_method. Choose 'acf', 'pacf', or 'aic'.")

        test_result = granger_causality_test(target_series, var_series, max_lag)
        results.append((var, max_lag, test_result))

    results.sort(key=lambda x: x[2])  # Sort by Granger causality test result
    selected_vars = [(var, max_lag) for var, max_lag, _ in results[:num_vars_to_select]]

    return selected_vars

def select_lag_acf(series, max_lag=50):
    acf_vals = acf(series, nlags=max_lag)
    return np.argmax(np.abs(acf_vals[1:])) + 1

def select_lag_pacf(series, max_lag=50):
    pacf_vals = pacf(series, nlags=max_lag)
    return np.argmax(np.abs(pacf_vals[1:])) + 1

def select_lag_aic(target_series, var_series, max_lag=50):
    lags = range(1, max_lag + 1)
    aic_values = []

    for lag in lags:
        X = lagmat2ds(var_series, lag, trim='both', dropex=1)
        X = sm.add_constant(X)
        model = sm.OLS(target_series[lag:], X)
        result = model.fit()
        aic_values.append(result.aic)

    return lags[np.argmin(aic_values)]

def granger_causality_test(target_series, var_series, max_lag):
    X = lagmat2ds(var_series, max_lag, trim='both', dropex=1)
    X = sm.add_constant(X)
    y = target_series[max_lag:]
    model = sm.OLS(y, X)
    result = model.fit()

    return result.f_pvalue

def create_lagged_data(data, selected_vars):
    lagged_data = pd.DataFrame(index=data.index)
    for var, max_lag in selected_vars:
        lagged_values = lagmat2ds(data[var], max_lag, trim='both', dropex=1)
        for lag in range(1, max_lag + 1):
            lagged_data[f'{var}_lag{lag}'] = np.nan  # Initialize the column with NaN values
            lagged_data.loc[:, f'{var}_lag{lag}'].iloc[max_lag:] = lagged_values[:, lag]
        # Include the original series in lagged_data
        lagged_data[var] = data[var]
    return pd.concat([data[target_var], lagged_data], axis=1)

In [23]:
df4 = df3.copy()
df4 = df4[low_vif_variables["Variable"]]
selected_vars = select_variables_granger_causality(df4, max_lag_method='aic', num_vars_to_select=25)

In [34]:
final_data = create_lagged_data(df4, selected_vars)

Save a pkl object that contains everything needed for the backtest

In [39]:
class selected_variables_object:
  def __init__(self,final_data, target_var, initial_train_size,selected_vars):
    self.final_data = final_data
    self.target_var = target_var
    self.initial_train_size = initial_train_size
    self.selected_vars = selected_vars
    

selected_variables_10Y_UST_v2 = selected_variables_object(final_data, target_var, initial_train_size, selected_vars)
print(selected_variables_10Y_UST_v2)

import pickle

# Serialize the object using pickle and save to a file
with open('selected_variables_10Y_UST_v2.pkl', 'wb') as file:
    pickle.dump(selected_variables_10Y_UST_v2, file)